Given discretised hfev1 variable node, define the prior distribution from LMS model

In [2]:
import src.modelling_fev1.pred_fev1 as pred_fev1
import numpy as np
import src.models.helpers as mh
from scipy.stats import norm
import plotly.graph_objects as go

## Compare implementation with real example

In [3]:
pred_fev1.calc_predicted_FEV1_LMS_straight(107, 5, "Male", True)

M = exp(-10.342 + 2.2196*ln(107) + 0.0574*ln(5) + -0.0752420061)
S = exp(-2.3268 + 0.0798*ln(5) + 0.1534965724)
L = 0.8866 + 0.085*ln(5)
LMS pred for zscore -1.644 = exp(ln(1 + -1.644 * 1.0234022225568986 * 0.12939697398113087) / 1.0234022225568986 + ln(1.0480702447075994)) = 0.8245143022244273


{'M': 1.0480702447075994,
 'sigma_up': 0.13542047783272881,
 'sigma_down': 0.13583240112542327,
 'LLN': 0.8245143022244273,
 'L': 1.0234022225568986,
 'S': 0.12939697398113087}

In [4]:
# Young white boy
age = 4.8
sex="Male"
height=107
spline_vals = {
    "Mspline": -0.0766,
    "Sspline": 0.1581,
}
coeffs = pred_fev1.load_LMS_coeffs(sex)
print(coeffs)
pred_fev1.calc_predicted_FEV1_LMS(spline_vals, coeffs, height, age, sex, True)

{'M': {'Intercept': -10.342, 'Height': 2.2196, 'Age': 0.0574}, 'S': {'Intercept': -2.3268, 'Age': 0.0798}, 'L': {'Intercept': 0.8866, 'Age': 0.085}}
M = exp(-10.342 + 2.2196*ln(107) + 0.0574*ln(4.8) + -0.0766)
S = exp(-2.3268 + 0.0798*ln(4.8) + 0.1581)
L = 0.8866 + 0.085*ln(4.8)
LMS pred for zscore -1.644 = exp(ln(1 + -1.644 * 1.019932353022677 * 0.12957123786469144) / 1.019932353022677 + ln(1.0441983216227528)) = 0.8212568103815278


{'M': 1.0441983216227528,
 'sigma_up': 0.135130716328544,
 'sigma_down': 0.1354811997810803,
 'LLN': 0.8212568103815278,
 'L': 1.019932353022677,
 'S': 0.12957123786469144}

In [5]:
pred_fev1.calc_predicted_FEV1_LMS(
        pred_fev1.load_LMS_spline_vals(30, "Male"),
        pred_fev1.load_LMS_coeffs("Male"),
        170,
        30,
        "Male",
    )

{'M': 4.063772841379649,
 'sigma_up': 0.4839054202339641,
 'sigma_down': 0.49427898234144285,
 'LLN': 3.2447818928058143,
 'L': 1.1757017774412832,
 'S': 0.12028487464044268}

## Archive

In [7]:
# Define hfev1 variable node
HFEV1 = mh.VariableNode("HFEV1", 2, 6, 0.1, prior=None)
HFEV1.bins

array([2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2,
       3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5,
       4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8,
       5.9])

In [8]:
def calc_hfev1_prior(hfev1_bins, height, age, sex):
    """
    Computes the prior fo hfev1 given its bins
    This model uses the inversed LMS function to compute the zscores of each bin given an array of hfev1/predictedFEV1 bin values
    """
    # Compute the predicted FEV1 for the individual
    fev1 = pred_fev1.calc_predicted_FEV1_LMS_straight(height, age, sex)
    S = fev1["S"]
    M = fev1["mean"]
    L = fev1["L"]

    # Define inverse LMS function
    def inverse_lms_pred_fev1(fev1_values, S, M, L):
        return (np.exp(L * np.log(fev1_values / M)) - 1) / (S * L)

    # Compute zscores for each bin
    zscores = inverse_lms_pred_fev1(hfev1_bins, S, M, L)

    # Get probabilities for each bin
    p = norm.pdf(zscores)
    p = p / p.sum()

    return p

In [9]:
# Validate inverse function
S = 2
L = 1
M = 4

z = 300
a = np.exp(np.log(1 + z * S * L) / L + np.log(M))
print(f"z={z}, a={a}")

z_bis = (np.exp(L * np.log(a / M)) - 1) / (S * L)
print(f"z_bis={z_bis}, a={a}")

z=300, a=2404.0000000000005
z_bis=300.0, a=2404.0000000000005


In [10]:
p, z = pred_fev1.calc_proba_for_hfev1(HFEV1.bins, 172, 30, "Male")
print(f"sum of lms p = {sum(p)}, len(p) = {len(p)}")

print(HFEV1.card)

p_gauss = norm(4.17, 0.45).pdf(HFEV1.bins)
# Normalise p_gauss
p_gauss = p_gauss / p_gauss.sum()
print(f"sum of p_gauss = {sum(p_gauss)}, len(p_gauss) = {len(p_gauss)}")


# Using plotly go, plot the probability distribution

fig = go.Figure()
# Add trace with values from normal distribution centered on 4.17 and sigma 0.45
fig.add_trace(
    go.Scatter(x=HFEV1.bins, y=p_gauss, mode="lines+markers", name="N(4.17, 0.45)")
)
fig.add_trace(go.Scatter(x=HFEV1.bins, y=p, mode="lines+markers", name="LMS hfev1"))
fig.update_layout(title="Probability distribution for hfev1")
fig.show()

AttributeError: module 'src.modelling_fev1.pred_fev1' has no attribute 'calc_proba_for_hfev1'

In [57]:
# Calculate area under curve for p
print(f"Area under curve for p = {np.trapz(p, HFEV1.bins)}")

Area under curve for p = 0.09999604386169679


In [6]:
HFEV1.get_bins_str()[0]

'[2.0; 2.1)'

In [10]:
last_elem = 6
bins_str = []
bins_arr = np.array([])
for i in range(HFEV1.card):
    low = round(HFEV1.bins[i], 2)
    up = HFEV1.b if i == (HFEV1.card - 1) else round(HFEV1.bins[i + 1], 2)

[
    f"[{round(HFEV1.bins[i], 2)}; {last_elem if i == (HFEV1.card-1) else round(HFEV1.bins[i+1], 2)})"
    for i in range(HFEV1.card)
]

['[2.0; 2.1)',
 '[2.1; 2.2)',
 '[2.2; 2.3)',
 '[2.3; 2.4)',
 '[2.4; 2.5)',
 '[2.5; 2.6)',
 '[2.6; 2.7)',
 '[2.7; 2.8)',
 '[2.8; 2.9)',
 '[2.9; 3.0)',
 '[3.0; 3.1)',
 '[3.1; 3.2)',
 '[3.2; 3.3)',
 '[3.3; 3.4)',
 '[3.4; 3.5)',
 '[3.5; 3.6)',
 '[3.6; 3.7)',
 '[3.7; 3.8)',
 '[3.8; 3.9)',
 '[3.9; 4.0)',
 '[4.0; 4.1)',
 '[4.1; 4.2)',
 '[4.2; 4.3)',
 '[4.3; 4.4)',
 '[4.4; 4.5)',
 '[4.5; 4.6)',
 '[4.6; 4.7)',
 '[4.7; 4.8)',
 '[4.8; 4.9)',
 '[4.9; 5.0)',
 '[5.0; 5.1)',
 '[5.1; 5.2)',
 '[5.2; 5.3)',
 '[5.3; 5.4)',
 '[5.4; 5.5)',
 '[5.5; 5.6)',
 '[5.6; 5.7)',
 '[5.7; 5.8)',
 '[5.8; 5.9)',
 '[5.9; 6)']